In [1]:
import numpy as np
from pyscf import gto, scf, fci
from tabulate import tabulate
from moha.hamiltonians import HamHub
from utils import Vee, generate_adjacency_matrix
from RDMFS import MU_U2RDM
import inspect 
import scipy 
#from numba.openmp import openmp_context as openmp

In [2]:
inspect.getfile(HamHub)

'/home/jerhard/1RDMFT/lib/python3.12/site-packages/moha/hamiltonians.py'

This script produces and prints the matrices that represent the Hubbard Model Hamiltonian. The point was to look at some details of it so it can be better understood—it's a showcase. The systems used as examples are rings, i.e., there are periodic boundary conditions. See additional comments on the matrices in the following boxes.

In [3]:
N = 2
norbs = N
nelec = N
connectivity = generate_adjacency_matrix(N)
U=2
t=-1
E0=2
hubbard = HamHub(connectivity,alpha=t, beta=E0, u_onsite=U*np.ones((N)),
                      sym=8)

h_0 = hubbard.generate_zero_body_integral()
h_1 = hubbard.generate_one_body_integral(basis='spinorbital basis', dense=True)
h2 = hubbard.generate_two_body_integral(sym=8,basis='spinorbital basis',
                                                dense=True)

h2 = np.transpose(h2, (0, 2, 1, 3))
h2_hub_ab = h2[0:N,0:N,N:2*N,N:2*N]
h2_hub_aa = h2[0:N,0:N,0:N,0:N]
h2_hub_bb = h2[N:2*N,N:2*N,N:2*N,N:2*N]
h_1 = (h_1[0:N,0:N], h_1[N:2*N,N:2*N])
h2_hub = (h2_hub_aa, h2_hub_ab, h2_hub_bb)



In [4]:
np.where(h2 != 0)

(array([0, 1, 2, 3]),
 array([0, 1, 2, 3]),
 array([2, 3, 0, 1]),
 array([2, 3, 0, 1]))

In [5]:
print(h_1[0])

[[-1.  2.]
 [ 2. -1.]]


In [6]:
N = 4
norbs = N
nelec = N
connectivity = generate_adjacency_matrix(N)
U=2
t=-1
E0=2
hubbard = HamHub(connectivity,alpha=t, beta=E0, u_onsite=U*np.ones((N)),
                      sym=8)

h_0 = hubbard.generate_zero_body_integral()
h_1 = hubbard.generate_one_body_integral(basis='spinorbital basis', dense=True)
h2 = hubbard.generate_two_body_integral(sym=8,basis='spinorbital basis',
                                                dense=True)

h2 = np.transpose(h2, (0, 2, 1, 3))
h2_hub_ab = h2[0:N,0:N,N:2*N,N:2*N]
h2_hub_aa = h2[0:N,0:N,0:N,0:N]
h2_hub_bb = h2[N:2*N,N:2*N,N:2*N,N:2*N]
h_1 = (h_1[0:N,0:N], h_1[N:2*N,N:2*N])
h2_hub = (h2_hub_aa, h2_hub_ab, h2_hub_bb)

Thw following operator is:

\begin{eqnarray}
    \hat n_{p,\uparrow} \hat n_{p,\downarrow} = \hat a_{p, \uparrow}^{\dagger} \hat a_{p, \uparrow} \hat a_{p, \downarrow}^{\dagger} \hat a_{p, \downarrow} = - \hat a_{p, \uparrow}^{\dagger} \hat a_{p, \downarrow}^{\dagger} \hat a_{p, \uparrow} \hat a_{p, \downarrow}
\end{eqnarray}

which can be interpreted as, is a pair of up and down spin at location $p$, both in the Bra and in the Ket state.

The elements of the $h_2$ would correspondingly be:

\begin{eqnarray}
    h_{2_{ijkl}} = \delta_{ik} \delta_{jl} \delta_{i(j-M)}
\end{eqnarray}
where $M$ is the basis set size and correspondingly $i+M$ is $i, \downarrow$ to any $i, \uparrow$

assuming the index $ijkl$ counts through the whole basis Homogeneously. 

In [7]:
np.where(h2 != 0)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([4, 5, 6, 7, 0, 1, 2, 3]),
 array([4, 5, 6, 7, 0, 1, 2, 3]))

This component
\begin{eqnarray}
    t \sum_{p,q, p \neq q}^n  \hat a^{\dagger}_p \hat  a_q
\end{eqnarray}
can be interpreted as a particle goes from site $p$ to site $q$. It is the kinetic energy in this picture, as matrix it will be:
\begin{eqnarray}
     t_{ij} = (1 - \delta_{ij}) \delta_{i(j-1)} + (1 - \delta_{ij}) \delta_{i(j+1)}
\end{eqnarray}
and $\delta_{i(j-1)}, \delta_{i(j+1)}$ makes sure only nearest neighbors are taken for jumping. 
    

In [8]:
print(h_1[0])

[[-1.  2.  0.  2.]
 [ 2. -1.  2.  0.]
 [ 0.  2. -1.  2.]
 [ 2.  0.  2. -1.]]
